In [3]:
#MobileNet+Densenet+Manual On Stacked CLassifeirs

from sklearn.ensemble import StackingClassifier
import numpy as np
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# Load features
mobilenet = np.load("featuresets/mobilenet_features.npy")
densenet = np.load("featuresets/densenet_features.npy")
manual = np.load("featuresets/manual_features.npy")
labels = np.load("featuresets/vgg_labels.npy")

# Combine features
combined = np.hstack((mobilenet, densenet, manual))

# Handle missing values
imputer = SimpleImputer(strategy="mean")
combined = imputer.fit_transform(combined)

# Scale
scaler = StandardScaler()
combined = scaler.fit_transform(combined)

# Split
X_train, X_test, y_train, y_test = train_test_split(combined, labels, test_size=0.2, random_state=42, stratify=labels)

# Define base learners
base_learners = [
    ('svm', SVC(probability=True, random_state=42)),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)),
    ('lr', LogisticRegression(max_iter=1000, random_state=42))
]

# Meta-learner
meta_learner = LogisticRegression(max_iter=1000, random_state=42)

# Stacking classifier
stacked_model = StackingClassifier(estimators=base_learners, final_estimator=meta_learner, cv=5)

# Train and evaluate
stacked_model.fit(X_train, y_train)
y_pred = stacked_model.predict(X_test)

print("\n[Stacked Model]")
print(classification_report(y_test, y_pred))
print("Stacked Accuracy:", accuracy_score(y_test, y_pred))


D:\Anaconda\Lib\site-packages\xgboost\training.py:183: UserWarning: [12:36:53] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
D:\Anaconda\Lib\site-packages\xgboost\training.py:183: UserWarning: [12:47:55] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
D:\Anaconda\Lib\site-packages\xgboost\training.py:183: UserWarning: [12:49:40] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
D:\Anaconda\Lib\site-packages\xgboost\training.py:183: UserWarning: [12:51:38] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
D:\Anaconda\Lib\site-pac


[Stacked Model]
              precision    recall  f1-score   support

           0       0.68      0.64      0.66       200
           1       0.94      0.94      0.94       200
           2       0.69      0.68      0.68       200
           3       0.87      0.86      0.87       200
           4       0.89      0.89      0.89       200
           5       0.69      0.69      0.69       200
           6       0.65      0.70      0.67       200

    accuracy                           0.77      1400
   macro avg       0.77      0.77      0.77      1400
weighted avg       0.77      0.77      0.77      1400

Stacked Accuracy: 0.7714285714285715


In [9]:
# XGBoost as base learner
base_learners = [
    ('svm', SVC(probability=True, random_state=42)),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)),
    ('lr', LogisticRegression(max_iter=1000, random_state=42))
]

# Meta-learner: XGBoost
meta_learner = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)

# Stacked classifier
stacked_model = StackingClassifier(
    estimators=base_learners,
    final_estimator=meta_learner,
    cv=5,
    passthrough=False,  # change to True if you want original features passed to meta too
    n_jobs=-1
)

# Train and evaluate
stacked_model.fit(X_train, y_train)
y_pred = stacked_model.predict(X_test)

print("\n[Stacked Model with XGBoost Meta-Learner]")
print(classification_report(y_test, y_pred))
print("Stacked Accuracy:", accuracy_score(y_test, y_pred))

D:\Anaconda\Lib\site-packages\xgboost\training.py:183: UserWarning: [14:22:33] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



[Stacked Model with XGBoost Meta-Learner]
              precision    recall  f1-score   support

           0       0.66      0.56      0.61       200
           1       0.93      0.92      0.92       200
           2       0.58      0.60      0.59       200
           3       0.83      0.85      0.84       200
           4       0.88      0.85      0.87       200
           5       0.66      0.65      0.65       200
           6       0.56      0.65      0.60       200

    accuracy                           0.72      1400
   macro avg       0.73      0.72      0.73      1400
weighted avg       0.73      0.72      0.73      1400

Stacked Accuracy: 0.725


In [5]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.metrics import classification_report, accuracy_score

# Load features
mobilenet = np.load("featuresets/mobilenet_features.npy")
densenet = np.load("featuresets/densenet_features.npy")
manual = np.load("featuresets/manual_features.npy")
labels = np.load("featuresets/vgg_labels.npy")

# Combine
combined = np.hstack((mobilenet, densenet, manual))
print("Combined shape before selection:", combined.shape)

# Handle NaNs
imputer = SimpleImputer(strategy="mean")
combined = imputer.fit_transform(combined)

# Normalize
scaler = StandardScaler()
combined = scaler.fit_transform(combined)

# Select top 1000 features using mutual information
selector = SelectKBest(mutual_info_classif, k=1000)
combined = selector.fit_transform(combined, labels)
print("Combined shape after SelectKBest:", combined.shape)

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    combined, labels, test_size=0.2, random_state=42, stratify=labels
)


Combined shape before selection: (7000, 2322)
Combined shape after SelectKBest: (7000, 1000)


In [7]:
# Define base learners
base_learners = [
    ('svm', SVC(probability=True, random_state=42)),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)),
    ('lr', LogisticRegression(max_iter=1000, random_state=42))
]

# Meta-learner
meta_learner = LogisticRegression(max_iter=1000, random_state=42)

# Stacking classifier
stacked_model = StackingClassifier(estimators=base_learners, final_estimator=meta_learner, cv=5)

# Train and evaluate
stacked_model.fit(X_train, y_train)
y_pred = stacked_model.predict(X_test)

print("\n[Stacked Model]")
print(classification_report(y_test, y_pred))
print("Stacked Accuracy:", accuracy_score(y_test, y_pred))

D:\Anaconda\Lib\site-packages\xgboost\training.py:183: UserWarning: [13:57:23] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
D:\Anaconda\Lib\site-packages\xgboost\training.py:183: UserWarning: [14:02:00] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
D:\Anaconda\Lib\site-packages\xgboost\training.py:183: UserWarning: [14:02:43] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
D:\Anaconda\Lib\site-packages\xgboost\training.py:183: UserWarning: [14:03:26] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
D:\Anaconda\Lib\site-pac


[Stacked Model]
              precision    recall  f1-score   support

           0       0.67      0.61      0.64       200
           1       0.93      0.94      0.93       200
           2       0.65      0.65      0.65       200
           3       0.87      0.84      0.86       200
           4       0.88      0.88      0.88       200
           5       0.67      0.68      0.67       200
           6       0.61      0.67      0.64       200

    accuracy                           0.75      1400
   macro avg       0.75      0.75      0.75      1400
weighted avg       0.75      0.75      0.75      1400

Stacked Accuracy: 0.7521428571428571


In [11]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier

# Load features
mobilenet = np.load("featuresets/mobilenet_features.npy")
densenet = np.load("featuresets/densenet_features.npy")
manual = np.load("featuresets/manual_features.npy")
labels = np.load("featuresets/vgg_labels.npy")

# Combine features
combined = np.hstack((mobilenet, densenet, manual))
print("Combined feature shape:", combined.shape)

# Handle missing values
imputer = SimpleImputer(strategy="mean")
combined = imputer.fit_transform(combined)

# Scale features
scaler = StandardScaler()
combined_scaled = scaler.fit_transform(combined)

# Apply PCA
pca = PCA(n_components=500)
X_pca = pca.fit_transform(combined_scaled)
print("PCA-transformed shape:", X_pca.shape)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_pca, labels, test_size=0.2, random_state=42, stratify=labels)

# Define base and meta models
base_models = [
    ('svm', SVC(probability=True, random_state=42)),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)),
    ('lr', LogisticRegression(max_iter=1000, random_state=42))
]

meta_model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)

# Stacking classifier
stacked_model = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=5)

# Train
stacked_model.fit(X_train, y_train)

# Predict & evaluate
y_pred = stacked_model.predict(X_test)
print("\n[PCA + Stacked Model]")
print(classification_report(y_test, y_pred))
print("Stacked Accuracy:", accuracy_score(y_test, y_pred))


Combined feature shape: (7000, 2322)
PCA-transformed shape: (7000, 500)


D:\Anaconda\Lib\site-packages\xgboost\training.py:183: UserWarning: [14:40:09] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
D:\Anaconda\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
D:\Anaconda\Lib\site-packages\xgboost\training.py:183: UserWarning: [14:42:58] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
D:\Anaconda


[PCA + Stacked Model]
              precision    recall  f1-score   support

           0       0.65      0.54      0.59       200
           1       0.91      0.93      0.92       200
           2       0.60      0.67      0.63       200
           3       0.83      0.80      0.81       200
           4       0.86      0.86      0.86       200
           5       0.66      0.62      0.64       200
           6       0.62      0.70      0.66       200

    accuracy                           0.73      1400
   macro avg       0.73      0.73      0.73      1400
weighted avg       0.73      0.73      0.73      1400

Stacked Accuracy: 0.73


In [13]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score

# Load features and labels
mobilenet = np.load("featuresets/mobilenet_features.npy")
densenet = np.load("featuresets/densenet_features.npy")
manual = np.load("featuresets/manual_features.npy")
labels = np.load("featuresets/vgg_labels.npy")

# Combine features
combined = np.hstack((mobilenet, densenet, manual))

# Handle missing values
imputer = SimpleImputer(strategy="mean")
combined = imputer.fit_transform(combined)

# Normalize
scaler = StandardScaler()
combined = scaler.fit_transform(combined)

# Apply PCA (try n_components=500 to start)
pca = PCA(n_components=500, random_state=42)
combined_pca = pca.fit_transform(combined)
print("PCA reduced shape:", combined_pca.shape)

# Split
X_train, X_test, y_train, y_test = train_test_split(
    combined_pca, labels, test_size=0.2, random_state=42, stratify=labels
)

# Define base classifiers
base_learners = [
    ('svm', SVC(probability=True, random_state=42)),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)),
    ('lr_base', LogisticRegression(max_iter=1000, random_state=42))
]

# Meta classifier
meta_learner = LogisticRegression(max_iter=1000, random_state=42)

# Stacked model
stacked_model = StackingClassifier(estimators=base_learners, final_estimator=meta_learner, cv=5)
stacked_model.fit(X_train, y_train)

# Evaluate
y_pred = stacked_model.predict(X_test)
print("\n[Stacked Model with PCA]")
print(classification_report(y_test, y_pred))
print("Stacked PCA Accuracy:", accuracy_score(y_test, y_pred))


PCA reduced shape: (7000, 500)


D:\Anaconda\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:45:51] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
D:\Anaconda\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
D:\Anaconda\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:48:36] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
D:\Anaconda


[Stacked Model with PCA]
              precision    recall  f1-score   support

           0       0.68      0.59      0.63       200
           1       0.93      0.92      0.92       200
           2       0.66      0.68      0.67       200
           3       0.85      0.85      0.85       200
           4       0.89      0.89      0.89       200
           5       0.70      0.69      0.69       200
           6       0.63      0.70      0.66       200

    accuracy                           0.76      1400
   macro avg       0.76      0.76      0.76      1400
weighted avg       0.76      0.76      0.76      1400

Stacked PCA Accuracy: 0.7614285714285715
